# Full Search

## Load Bible Text

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from hanziconv import HanziConv
from data import read_bible

unv = read_bible('data/dnstrunv.tgz')
unv['text_s'] = unv.text.apply(HanziConv.toSimplified)

## Tokenize

In [3]:
import jieba
import jieba.posseg as pseg

jieba.load_userdict('bible_terms.txt')

unv['text_tk'] = unv.text_s.apply(pseg.lcut)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/w0/lx0qzxln5p72qnrg392g7nyc0000gn/T/jieba.cache
Loading model cost 2.007 seconds.
Prefix dict has been built successfully.


## Search

In [4]:
from typing import Iterable

def highlight_occurances(text: str, keywords: Iterable[str]) -> str:
    for i, kw in enumerate(keywords):
        text = text.replace(kw, highlight(kw, i))
    return text

def highlight(text: str, color_code: int) -> str:
    return f'\x1b[6;30;4{color_code + 1}m{text}\x1b[0m'

test_data = ['first', 'second', 'thrid', 'forth', 'fifth', 'sixth', 'seventh']
print(highlight_occurances(', '.join(test_data), test_data))

first, second, thrid, forth, fifth, sixth, seventh


In [41]:
from search import sentence_similarity, load_word_embeddings

load_word_embeddings('data/mini.h5')

In [42]:
searches = ['欢乐 祈祷', '挂虑 祈祷', '虑 祈祷', '喜乐 事奉', '求救', '信心 行事']

for search_term in searches:
    search_tk = pseg.lcut(search_term)
    print(f'Search for {search_term} ({search_tk}):')
    match_scores = {}
    for v, vers_tk in zip(unv.text_s, unv.text_tk):
        similarity = sentence_similarity(search_tk, vers_tk)
        match_kw = [kw for kw in similarity.keys() if similarity[kw] > 0]
        vers = highlight_occurances(v, match_kw)
        score = sum(similarity.values())
        match_scores[vers] = score
    for top_match in sorted(match_scores, key=match_scores.get, reverse=True)[:10]:
        print(f'Match: {match_scores[top_match]:7.4f} Verse: {top_match}')
    print()

Search for 欢乐 祈祷 ([pair('欢乐', 'a'), pair(' ', 'x'), pair('祈祷', 'v')]):
Match:  1.8144 Verse: 恶人献祭，为耶和华所憎恶；正直人祈祷，为他所喜悦。
Match:  1.7942 Verse: 撒母耳不喜悦他们说「立一个王治理我们」，他就祷告耶和华。
Match:  1.7942 Verse: 他祷告神，神就喜悦他，使他欢唿朝见神的面；神又看他为义。
Match:  1.7777 Verse: 我必领他们到我的圣山，使他们在祷告我的殿中喜乐。他们的燔祭和平安祭，在我坛上必蒙悦纳，因我的殿必称为万民祷告的殿。
Match:  1.7777 Verse: 你们中间有受苦的呢，他就该祷告；有喜乐的呢，他就该歌颂。
Match:  1.7777 Verse: 在指望中要喜乐；在患难中要忍耐；祷告要恆切。
Match:  1.7091 Verse: 哈拿祷告说：我的心因耶和华快乐；我的角因耶和华高举。我的口向仇敌张开；我因耶和华的救恩欢欣。
Match:  1.4742 Verse: 在那里，耶和华─你们神的面前，你们和你们的家属都可以吃，并且因你手所办的一切事蒙耶和华─你的神赐福，就都欢乐。
Match:  1.4742 Verse: 在耶和华所选择的地方，你当向耶和华─你的神守节七日；因为耶和华─你神在你一切的土产上和你手里所办的事上要赐福与你，你就非常地欢乐。
Match:  1.4094 Verse: 将要灭亡的为我祝福；我也使寡妇心中欢乐。

Search for 挂虑 祈祷 ([pair('挂虑', 'v'), pair(' ', 'x'), pair('祈祷', 'v')]):
Match:  1.9797 Verse: 应当一无挂虑，只要凡事借着祷告、祈求，和感谢，将你们所要的告诉神。
Match:  1.5609 Verse: 祷告完了，就起来，到门徒那里，见他们因为忧愁都睡着了，
Match:  1.5530 Verse: 哈拿心里愁苦，就痛痛哭泣，祈祷耶和华，
Match:  1.5131 Verse: 次日清早，他们起来，在耶和华面前敬拜，就回拉玛。到了家里，以利加拿和妻哈拿同房，耶和华顾念哈拿，
Match:  1.3913 Verse: 当时，部百流的父亲患热病